In [1]:
import pandas as pd
df = pd.read_csv(r'd:\DAP391m\Assigment final\Assignment_DAP - Copy - Copy\balance_data_way3.csv')

df.tail()

,text,label
6050,đồng đội.,1
6051,cải thiện sức khỏe.,0
6052,ban công ngắm thành phố.,0
6053,chẳng đẹp cả.,2
6054,điện thoại này.,1


In [2]:
# Kiểm tra trùng lặp dựa trên một tập hợp cột nhất định, ví dụ các cột
num_duplicates_columns = df.duplicated(subset=['text']).sum()
print(f"Số lượng hàng trùng lặp dựa trên các cột đã chọn: {num_duplicates_columns}")

Số lượng hàng trùng lặp dựa trên các cột đã chọn: 150


In [3]:
# Loại bỏ các hàng trùng lặp
data = df.drop_duplicates()

print(f"Số lượng hàng sau khi loại bỏ trùng lặp: {len(data)}")

Số lượng hàng sau khi loại bỏ trùng lặp: 5926


In [4]:
print(data.isnull().sum())

text     3
label    0
dtype: int64


In [5]:
data = data.dropna(subset=['text'])

In [6]:
data['label'].value_counts()

label
1    1993
2    1982
0    1948
Name: count, dtype: int64

In [7]:
print(data.isnull().sum())

text     0
label    0
dtype: int64


In [8]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import RobertaForSequenceClassification, AutoTokenizer,AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment",num_labels=3, ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--wonrax--phobert-base-vietnamese-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [10]:
sample_data = ["Tui rất yêu bạn, tui thích bạn "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[0, 17410, 59, 413, 18964, 1301, 4, 6265, 543, 88, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [11]:
X = list(data["text"])
y = list(data["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [12]:
len(y_train)

5034

In [13]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
print(class_weights)

[1.01389728 1.         0.98647854]


In [14]:
# Print the calculated class weights
for class_id, weight in enumerate(class_weights):
    print(f"Class {class_id}: Weight {weight:.2f}")

Class 0: Weight 1.01
Class 1: Weight 1.00
Class 2: Weight 0.99


In [15]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [16]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [17]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
len(X_train),len(X_val)

(5034, 889)

In [19]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])
    

In [20]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [21]:
train_dataset[5]

{'input_ids': tensor([   0, 2404, 6928,  286, 1783, 1593, 1818, 2951,  944,    2,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor(1)}

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    precision = precision_score(y_true=labels, y_pred=predictions, average='weighted')
    recall = recall_score(y_true=labels, y_pred=predictions, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }

In [23]:
from transformers import TrainingArguments, Trainer
import transformers
trainer = Trainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        args= TrainingArguments(
            output_dir='./results',
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            # gradient_accumulation_steps=1,
            warmup_steps=10,
            num_train_epochs=3,
            logging_steps=100,
            report_to="none",
            eval_strategy="steps",
            save_strategy="steps",
            save_total_limit=10,
            load_best_model_at_end=True
        ),
        data_collator= transformers.DataCollatorWithPadding(tokenizer=tokenizer,  padding = True, return_tensors="pt")
    )

In [ ]:
trainer.train()

  0%|          | 0/1890 [00:00<?, ?it/s]

In [ ]:
trainer.evaluate()

  0%|          | 0/112 [00:00<?, ?it/s]

{'eval_loss': 0.6961881518363953,
 'eval_accuracy': 0.7412823397075365,
 'eval_precision': 0.7431741259767478,
 'eval_recall': 0.7412823397075365,
 'eval_f1_score': 0.740719942428645,
 'eval_runtime': 35.085,
 'eval_samples_per_second': 25.338,
 'eval_steps_per_second': 3.192,
 'epoch': 3.0}

In [ ]:
text = "người buồn thì cảnh có vui bao giờ"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to(device)  # Chuyển inputs sang thiết bị hiện tại
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions
#{0: "Other", 1: "Disgust", 2:"Enjoyment", 3:"Anger", 4:"Surprise", 5:"Sadness", 6:"Fear"}

SequenceClassifierOutput(loss=None, logits=tensor([[-1.3411, -1.1394,  2.8116]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.0152, 0.0186, 0.9662]], grad_fn=<SoftmaxBackward0>)


array([[0.01519036, 0.01858436, 0.9662252 ]], dtype=float32)

In [ ]:
trainer.save_model(r'd:\DAP391m\Assigment final\Assignment_DAP - Copy - Copy\model_emotion\save_weight_1')

In [ ]:
#import torch
# Save your model 
#torch.save(model.state_dict(), r'C:\Users\admin\Downloads\Assignment_DAP\model_emotion\custome_model2.pth')

In [ ]:
from transformers import RobertaForSequenceClassification, AutoTokenizer,AutoModelForSequenceClassification
import torch

model_2 = RobertaForSequenceClassification.from_pretrained(r'd:\DAP391m\Assigment final\Assignment_DAP - Copy - Copy\model_emotion\save_weight_1')
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

In [ ]:
# text = "That was good point"
text = "thật là cảm động quá đi á"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.03785626, 0.9447088 , 0.01743495]], dtype=float32)

In [ ]:
def change_format_to(list_input):
    formatted_list = [[f"{value:.3f}" for value in sublist] for sublist in list_input]
    return formatted_list

text = ":) anh buồn em nhiều lắm"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
output_list = change_format_to(predictions)

print(output_list)

[['0.011', '0.014', '0.975']]


In [ ]:
import plotly.express as px


# Convert strings to floats and scale up by 100
scaled_data = [float(value) * 100 for value in output_list[0]]

# Custom names for each bar
bar_names = ['Other', 'Enjoyment', 'Sadness']

# Create a DataFrame for the plot
import pandas as pd
df = pd.DataFrame({'Kind of Emotion': bar_names, 'Data': output_list[0], 'Scaled Value': scaled_data})

# Create the interactive bar chart using Plotly Express
fig = px.bar(df, y='Kind of Emotion', x='Scaled Value', text='Scaled Value', labels={'Scaled Value': 'Probability'})

# Add title and customize layout
# Add title and customize layout
fig.update_layout(
    title= text ,
    xaxis_title='Probability',
    yaxis_title='Kind of Emotion',
    width=800,  # Set the width of the chart
    height=400  # Set the height of the chart
)


# Show the interactive plot in Jupyter Notebook
fig.show()